# <center>$Fine\ Tuning - LoRA$

In [2]:
pip install torch torchvision datasets


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install peft

  Using cached peft-0.17.1-py3-none-any.whl.metadata (14 kB)
  Using cached transformers-4.55.3-py3-none-any.whl.metadata (41 kB)
  Using cached accelerate-1.10.0-py3-none-any.whl.metadata (19 kB)
Using cached peft-0.17.1-py3-none-any.whl (504 kB)
Using cached accelerate-1.10.0-py3-none-any.whl (374 kB)
Using cached transformers-4.55.3-py3-none-any.whl (11.3 MB)

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
import transformers, peft

print(transformers.__version__)
print(peft.__version__)

4.55.3
0.17.1


In [62]:
import torch

print(torch.backends.mps.is_available())
print(torch.backends.mps.is_built())  

True
True


In [79]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### <center>$LLM - Gemma3:270M$

##### 1. $Pipeline$

In [33]:
from transformers import pipeline

In [63]:
model = pipeline(
    model = 'google/gemma-3-270m',
    device = 'mps'
)
print(model("Hello, How are you?"))

Device set to use mps


[{'generated_text': 'Hello, How are you? I hope you are doing well. I\'m a 24-year-old student at the University of New Mexico in Albuquerque, NM. I\'m a software engineer with a major in Computer Science. I like to work on projects that involve technology and software engineering. I\'m looking forward to learning more about these subjects in the future. I\'m excited to meet you!\n\n<h2>Quotes</h2>\n\n* <h3>t.</h3>\n\n  "The problem of the day is that it\'s not about you, but it\'s about you doing it. This is the most important thing you can do for yourself." - David Attenborough\n\n  "The greatest enemy of success is not your own failures, but your own failures together with your own failures." - Richard Branson\n\n  "We are all going to have to learn to live with the fact that our lives have no purpose, that our lives are just a waste of time and money." - Seth Godin\n\n  "The problem of the day is that it\'s not about you, but it\'s about you doing it." - David Attenborough\n\n  "Th

##### 1.2 $Downloading\ the\  Model$

In [144]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [145]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")

In [146]:
#torch_dtype = torch.float16 (For GPU)
#Mac - MPS(Metal Performance Shaders)
#CPU  = gemma3.to('cpu')
gemma3 = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m", attn_implementation="eager").to('mps')

In [147]:
print(gemma3)

Gemma3ForCausalLM(
  (model): Gemma3TextModel(
    (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x Gemma3DecoderLayer(
        (self_attn): Gemma3Attention(
          (q_proj): Linear(in_features=640, out_features=1024, bias=False)
          (k_proj): Linear(in_features=640, out_features=256, bias=False)
          (v_proj): Linear(in_features=640, out_features=256, bias=False)
          (o_proj): Linear(in_features=1024, out_features=640, bias=False)
          (q_norm): Gemma3RMSNorm((256,), eps=1e-06)
          (k_norm): Gemma3RMSNorm((256,), eps=1e-06)
        )
        (mlp): Gemma3MLP(
          (gate_proj): Linear(in_features=640, out_features=2048, bias=False)
          (up_proj): Linear(in_features=640, out_features=2048, bias=False)
          (down_proj): Linear(in_features=2048, out_features=640, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma3RMSNorm((640,), eps

In [181]:
import torch
import torch.nn as nn

layer = nn.Linear(in_features=2, out_features=4) #Linear layer


x = torch.tensor([[1.0, 2.0]])
y = layer(x)

print(y)

tensor([[-0.4273, -1.1284,  0.7985,  0.3851]], grad_fn=<AddmmBackward0>)


### <center>$2.Dataset\ Preparation$

#### $ 2.1\ Convert\ TXT\ into\ JSON\ data$

In [10]:
from datasets import load_dataset, Dataset
import json

data = load_dataset('text', data_files = 'FineTuning_data.txt')
print(data)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 563
    })
})


In [11]:
lines = [x['text'].strip() for x in data['train'] if x['text'].strip()]
print(lines[2])

answer: I am Lila


In [12]:
examples = []

for i in range(0, len(lines), 3):
    context = lines[i].replace("context:", "").strip()
    question = lines[i+1].replace("question:", "").strip()
    answer = lines[i+2].replace("answer:", "").strip()

    examples.append({
        "prompt": question,
        "completion": context
    })

In [13]:
print(examples[0])

{'prompt': 'What is your name?', 'completion': 'I am a Large Language Model trained on large amount of text data, my name is Lila created by talented Data scientist named Kabil.'}


In [14]:
ft_data = Dataset.from_list(examples)
print(ft_data[0])

{'prompt': 'What is your name?', 'completion': 'I am a Large Language Model trained on large amount of text data, my name is Lila created by talented Data scientist named Kabil.'}


In [15]:
ft_data.to_json("FineTuning_data.json")
print("Data save into JSON format")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Data save into JSON format


----------------

#### $2.2\ Loading\ JSON\ data$

In [121]:
json_data = load_dataset('json', data_files = 'FineTuning_data.json')
print(json_data)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 141
    })
})


In [122]:
print(json_data['train'][100])

{'prompt': 'What type of learning is used to find hidden patterns in data without labeled responses?', 'completion': 'Unsupervised learning is used to find hidden patterns or intrinsic structures in input data without labeled responses.'}


#### $2.3\ Tokenization$

In [123]:
def preprocess(json_data):  
    json_data = json_data['prompt']+ "\n" +json_data['completion']

    tokens = tokenizer(
        json_data,
        max_length = 128,
        truncation = True,
        padding = 'max_length',
        padding_side = 'right'
    )
    tokens['labels'] = tokens['input_ids'].copy()

    return tokens

- Note: \
`prompt + completion + input_ids + attention_mask + labels` - Data Preprocessing format Casual LM Fine tuning

In [124]:
tokens = json_data.map(preprocess)
print(tokens)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 141
    })
})


`input_ids` - What goes into the Model \
`labels` - What the Model should predict \
`attention_mask` - Show which tokens are real (1 - $real$, 0 - $padded$)

In [174]:
print(tokens['train'][15])

{'prompt': 'What degrees does Kabil hold?', 'completion': "Kabil's educational background includes a Bachelor's degree in Mechanical Engineering followed by a Master's degree focusing on artificial intelligence. His academic journey has been marked by a strong performance in both his undergraduate and graduate studies.", 'input_ids': [2, 3689, 10674, 1677, 751, 6032, 2768, 236881, 107, 236855, 6032, 236789, 236751, 12524, 1695, 5132, 496, 32806, 236789, 236751, 5802, 528, 44813, 12464, 6641, 684, 496, 11930, 236789, 236751, 5802, 19541, 580, 16477, 14020, 236761, 4923, 13434, 9338, 815, 1010, 11373, 684, 496, 3188, 3736, 528, 1800, 914, 38024, 532, 17506, 5254, 236761, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

###  <center>$3. LoRA(Low\ Rank\ Apdatation)\ Tuning$

#### $3.1\ (PEFT)\ Parameter\ Efficient\ Fine\ Tuning$

In [148]:
from peft import LoraConfig, get_peft_model, TaskType

config = LoraConfig(
    task_type = TaskType.CAUSAL_LM,
    target_modules = ['q_proj', 'k_proj', 'v_proj', 'o_proj'] #Full Attention LoRA
)

model = get_peft_model(gemma3, config)

`q_proj` = $Query\ Projection$ \
`k_proj` = $Key\ Projection$ \
`v_proj` = $Value\ Projection$ \
`o_proj` = $Output\ Projection$

In [149]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForCausalLM(
      (model): Gemma3TextModel(
        (embed_tokens): Gemma3TextScaledWordEmbedding(262144, 640, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x Gemma3DecoderLayer(
            (self_attn): Gemma3Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=640, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=640, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_

#### $3.2\ Trainer$

##### $3.2.1\ Create\ Check-point$

In [150]:
from transformers import TrainingArguments, Trainer

In [151]:
train_args = TrainingArguments(
    output_dir = './results',
    num_train_epochs = 2,
    learning_rate = 0.001,
    logging_steps = 25,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    fp16 = False,
    report_to = "none"
)

In [152]:
trainer = Trainer(
    args = train_args,
    model = model,
    train_dataset = tokens['train']
)

In [153]:
trainer.train()

/Users/kabil/anaconda3/envs/PythonCourse/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
25,7.926400
50,4.497900


TrainOutput(global_step=72, training_loss=5.479748196072048, metrics={'train_runtime': 85.1949, 'train_samples_per_second': 3.31, 'train_steps_per_second': 0.845, 'total_flos': 21887884394496.0, 'train_loss': 5.479748196072048, 'epoch': 2.0})

##### $3.2.2 Restarter$

In [154]:
import time
from transformers import TrainingArguments, Trainer


train_args = TrainingArguments(
    #num_train_epochs = 10,
    output_dir = "./results",
    learning_rate = 0.001,
    logging_steps = 25,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 4,
    fp16 = False,
    report_to = "none"
)

total_epochs = 20
chunk_size = 2
rest_time = 60

for start_epoch in range(0, total_epochs, chunk_size):
    print(f"\n🚀 Training {chunk_size} epochs (from {start_epoch+1} to {start_epoch+chunk_size})...")

    train_args.num_train_epochs = start_epoch + chunk_size

    trainer = Trainer(
        args = train_args,
        model = model,
        train_dataset = tokens['train']
    )
    
    train_out = trainer.train(resume_from_checkpoint=True)
    print(train_out)

    print(f"😴 Resting for {rest_time} seconds to cool down...")
    time.sleep(rest_time)


🚀 Training 2 epochs (from 1 to 2)...


Step,Training Loss


TrainOutput(global_step=72, training_loss=0.0, metrics={'train_runtime': 0.0067, 'train_samples_per_second': 42404.68, 'train_steps_per_second': 10826.727, 'total_flos': 21887884394496.0, 'train_loss': 0.0, 'epoch': 2.0})
😴 Resting for 60 seconds to cool down...

🚀 Training 2 epochs (from 3 to 4)...


/Users/kabil/anaconda3/envs/PythonCourse/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
75,3.673500
100,3.350000
125,2.846400


...

Step,Training Loss
600,1.378300
625,1.290500


TrainOutput(global_step=648, training_loss=0.14645184410942924, metrics={'train_runtime': 72.4416, 'train_samples_per_second': 35.035, 'train_steps_per_second': 8.945, 'total_flos': 196990959550464.0, 'train_loss': 0.14645184410942924, 'epoch': 18.0})
😴 Resting for 60 seconds to cool down...

🚀 Training 2 epochs (from 19 to 20)...


/Users/kabil/anaconda3/envs/PythonCourse/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
650,1.205700
675,1.280000
700,1.227300


TrainOutput(global_step=720, training_loss=0.12462990979353587, metrics={'train_runtime': 76.8088, 'train_samples_per_second': 36.715, 'train_steps_per_second': 9.374, 'total_flos': 218878843944960.0, 'train_loss': 0.12462990979353587, 'epoch': 20.0})
😴 Resting for 60 seconds to cool down...


## $$Perplexity\ Calculation$$
`Take: 1`\
$Training\ Loss(TL) = 2.14$ \
$Epochs = 20$
$$ Perplexity = e^{(TL)} = 8.75 $$

`Take 2: (Full Attention)`\
$Training\ Loss(TL) = 1.22$ \
$Epochs = 20$
$$ Perplexity = e^{(TL)} = 3.38 $$

Note:
> Lower perplexity = better model (more confident and accurate).\
> `50` → very poor (random-like) \
> `10–20` → decent small model \
> `<5` → pretty good (fluent) \
> `~1` → near perfect 

### <center>$4.\ Saving\ the$ `Model`

In [155]:
trainer.save_model('./Tuner')
trainer.state.save_to_json("./Tuner/trainer_state.json")
tokenizer.save_pretrained('./Tuner')

('./Tuner/tokenizer_config.json',
 './Tuner/special_tokens_map.json',
 './Tuner/tokenizer.json')

#### $4.1\ Testing\ the$ `Model`

In [175]:
lila = pipeline(
    model = './Tuner',
    tokenizer = './Tuner',
    device = 'mps'
)

print(lila("What degrees does Kabil hold?")[0]['generated_text'])

Device set to use mps


What degrees does Kabil hold?
Kabil graduated from the B.E.T.M.M. (Business and Management Extension) in 2013. This degree is designed to provide practical experience in business and management.<unused5256>Diharchester University Club is a non-profit organization that raises funds for scholarships and projects. suichenthttps://www.wikipedia.org is a non-profit organization dedicated to preserving and promoting the achievements of people. Another way for your comment to be considered is to be associated with a non-profit organization such aschesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchesterchestercheste

----------------------------